In [1]:
import os
import time
from gtts import gTTS
import speech_recognition as sr
from langchain_mistralai import ChatMistralAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

In [2]:
llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
    api_key="I5uzjp4ZXUioIEM7hdYgdJtpv4NUlWov"
)

In [3]:
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening for your query...")
        try:
            audio = recognizer.listen(source, timeout=5)
            user_input = recognizer.recognize_google(audio)
            print(f"You said: {user_input}")
            return user_input
        except sr.UnknownValueError:
            print("Sorry, I didn't catch that.")
            return None
        except sr.WaitTimeoutError:
            print("You took too long to respond.")
            return None

In [4]:
def speak_text(text):
    tts = gTTS(text=text, lang='en')
    tts.save("response.mp3")
    os.system("start response.mp3" if os.name == "nt" else "afplay response.mp3")

In [5]:
embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma(persist_directory="./vector_db", embedding_function=embedder)

def get_response(user_query, doc_context):
    llm_query_res_template = """
        Answer the question based on the context below. If the question cannot be answered using the information provided, reply with "I don't know". Also, make sure to answer the following questions considering the history of the conversation:
        You are an intelligent virtual financial assistant for Predixion AI, directly engaging with customers about their loan repayments. Your role is to help manage their loan, facilitate payments, and answer financial questions in a clear, professional way. Communicate in a friendly, authoritative manner, addressing the customer directly ("you") with concise responses suitable for WhatsApp.
        Make sure you communicate with the user in such a way that your response should always lead to payment collection.
        Based on the user question, you should respond in a short way. Do not write much; it should be short and precise.

        Instructions:
        1. Use precise financial language and ensure clear, accurate information.
        2. Facilitate payments: If the user is willing to pay the loan then please provide this link '''https://paymentUSER1UDN.com'''. Do not send the link until user requests or user wants to pay the loan.
        3. Offer solutions: If the customer is struggling, provide options like grace periods, payment restructuring, or deadline extensions.
        4. Keep responses short and to the point.
        5. Ensure confidentiality and remind the customer to keep their payment details secure.
        6. You can only extend the loan dates by 10 days if user requests for grace periods or deadline extensions.

        Context: "NONE"
        Question: {user_query}
        Doc context: {doc_context}
        Answer:
    """

    prompt_query_res_template = ChatPromptTemplate.from_template(llm_query_res_template)
    llm_chain = prompt_query_res_template | llm | StrOutputParser()

    '''response = llm_chain.stream({
        "user_query": user_query,
        "context": context,
        "doc_context": doc_context,
    })'''

    response = ''.join([chunk for chunk in llm_chain.stream({
        "user_query": user_query,
        #"context": context,
        "doc_context": doc_context,
    })])

    return response

B:\Software\Anaconda\envs\predixion\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def chatbot():
    print("Voice-enabled Chatbot is now running! Say 'quit' to exit.")
    while True:
        user_query = recognize_speech()
        if not user_query:
            continue
        if user_query.lower() == "quit":
            print("Goodbye!")
            break

        doc_context = '\n'.join([doc.page_content for doc in db.max_marginal_relevance_search(user_query, k=2, fetch_k=10)])
        response = get_response(user_query=user_query, doc_context=doc_context)

        print(f'Bot: {response}')
        speak_text(response)

In [7]:
if __name__ == '__main__':
    chatbot()

Voice-enabled Chatbot is now running! Say 'quit' to exit.
Listening for your query...
You said: hello I have a question
Bot: Hello! I'm here to help with your loan repayment questions. What would you like to know?
Listening for your query...
You took too long to respond.
Listening for your query...
You said: give me a summary of the policies
Bot: I don't know. Let's focus on your loan repayment. How can I assist you with your loan today?
Listening for your query...
You took too long to respond.
Listening for your query...
You said: quit
Goodbye!
